In [2]:
pip install progressbar

  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12075 sha256=a826b8ef71315abfa29664c211e3b72787f9f85e01a6436b1dca0a9c1e9e53e6
  Stored in directory: c:\users\dragew\appdata\local\pip\cache\wheels\2c\67\ed\d84123843c937d7e7f5ba88a270d11036473144143355e2747
Successfully built progressbar
Note: you may need to restart the kernel to use updated packages.


In [10]:
import sys, os
import requests
import selenium
from selenium import webdriver
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pickle, progressbar, json, glob, time
from tqdm import tqdm

In [ ]:
###### 날짜 저장 ##########
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

sleep_sec = 0.5

In [12]:
####### 언론사별 본문 위치 태그 파싱 함수 ###########
print('본문 크롤링에 필요한 함수를 로딩하고 있습니다...\n' + '-' * 100)
def crawling_main_text(url):

    req = requests.get(url)
    req.encoding = None
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # 연합뉴스
    if ('://yna' in url) | ('app.yonhapnews' in url): 
        main_article = soup.find('div', {'class':'story-news article'})
        if main_article == None:
            main_article = soup.find('div', {'class' : 'article-txt'})
            
        text = main_article.text
        
    # MBC 
    elif '//imnews.imbc' in url: 
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
        
    # 매일경제(미라클), req.encoding = None 설정 필요
    elif 'mirakle.mk' in url:
        text = soup.find('div', {'class' : 'view_txt'}).text
        
    # 매일경제, req.encoding = None 설정 필요
    elif 'mk.co' in url:
        text = soup.find('div', {'class' : 'art_txt'}).text
        
    # SBS
    elif 'news.sbs' in url:
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
    
    # KBS
    elif 'news.kbs' in url:
        text = soup.find('div', {'id' : 'cont_newstext'}).text
        
    # JTBC
    elif 'news.jtbc' in url:
        text = soup.find('div', {'class' : 'article_content'}).text
        
    # 그 외
    else:
        text == None
        
    return text.replace('\n','').replace('\r','').replace('<br>','').replace('\t','')
    
    
press_nm = 'KBS'

print('검색할 언론사 : {}'.format(press_nm))

본문 크롤링에 필요한 함수를 로딩하고 있습니다...
----------------------------------------------------------------------------------------------------
검색할 언론사 : KBS


In [14]:
############### 브라우저를 켜고 검색 키워드 입력 ####################
query = input('검색할 키워드  : ')
news_num = int(input('수집 뉴스의 수(숫자만 입력) : '))

print('\n' + '=' * 100 + '\n')

print('브라우저를 실행시킵니다(자동 제어)\n')
chrome_path = 'C:/chromedriver_win32/chromedriver.exe'
browser = webdriver.Chrome(chrome_path)

news_url = 'https://search.naver.com/search.naver?where=news&query={}'.format(query)
browser.get(news_url)
time.sleep(sleep_sec)

######### 언론사 선택 및 confirm #####################
print('설정한 언론사를 선택합니다.\n')

search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
search_opn_btn.click()
time.sleep(sleep_sec)

bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')

# 기준 두번 째(언론사 분류순) 클릭하고 오픈하기
press_tablist = bx_press.find_elements_by_xpath('.//div[@role="tablist" and @class="option"]/a')
press_tablist[1].click()
time.sleep(sleep_sec)

# 첫 번째 것(언론사 분류선택)
bx_group = bx_press.find_elements_by_xpath('.//div[@class="api_select_option type_group _category_select_layer"]/div[@class="select_wrap _root"]')[0]

press_kind_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[0]
press_kind_btn_list = press_kind_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')


for press_kind_btn in press_kind_btn_list:
    
    # 언론사 종류를 순차적으로 클릭(좌측)
    press_kind_btn.click()
    time.sleep(sleep_sec)
    
    # 언론사선택(우측)
    press_slct_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[1]
    # 언론사 선택할 수 있는 클릭 버튼
    press_slct_btn_list = press_slct_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')
    # 언론사 이름들 추출
    press_slct_btn_list_nm = [psl.text for psl in press_slct_btn_list]
    
    # 언론사 이름 : 언론사 클릭 버튼 인 딕셔너리 생성
    press_slct_btn_dict = dict(zip(press_slct_btn_list_nm, press_slct_btn_list))
    
    # 원하는 언론사가 해당 이름 안에 있는 경우
    # 1) 클릭하고
    # 2) 더이상 언론사분류선택 탐색 중지
    if press_nm in press_slct_btn_dict.keys():
        print('<{}> 카테고리에서 <{}>를 찾았으므로 탐색을 종료합니다'.format(press_kind_btn.text, press_nm))
        
        press_slct_btn_dict[press_nm].click()
        time.sleep(sleep_sec)
        
        break


검색할 키워드  : esg
수집 뉴스의 수(숫자만 입력) : 100


브라우저를 실행시킵니다(자동 제어)



<ipython-input-14-81f17d0ce072>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)


설정한 언론사를 선택합니다.



<ipython-input-14-81f17d0ce072>:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
<ipython-input-14-81f17d0ce072>:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')
C:\Users\dragew\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:425: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


<방송/통신> 카테고리에서 <KBS>를 찾았으므로 탐색을 종료합니다


In [15]:
################ 뉴스 크롤링 ########################

print('\n크롤링을 시작합니다.')
# ####동적 제어로 페이지 넘어가며 크롤링
news_dict = {}
idx = 1
cur_page = 1

pbar = tqdm(total=news_num ,leave = True)
    
while idx < news_num:

    table = browser.find_element_by_xpath('//ul[@class="list_news"]')
    li_list = table.find_elements_by_xpath('./li[contains(@id, "sp_nws")]')
    area_list = [li.find_element_by_xpath('.//div[@class="news_area"]') for li in li_list]
    a_list = [area.find_element_by_xpath('.//a[@class="news_tit"]') for area in area_list]
 
    for n in a_list[:min(len(a_list), news_num-idx+1)]:
        n_url = n.get_attribute('href')
        news_dict[idx] = {'title' : n.get_attribute('title'), 
                          'url' : n_url,
                          'text' : crawling_main_text(n_url)}
        
        idx += 1
        pbar.update(1)
        
    if idx < news_num:
        cur_page +=1

        pages = browser.find_element_by_xpath('//div[@class="sc_page_inner"]')
        next_page_url = [p for p in pages.find_elements_by_xpath('.//a') if p.text == str(cur_page)][0].get_attribute('href')

        browser.get(next_page_url)
        time.sleep(sleep_sec)
    else:
        pbar.close()
        
        print('\n브라우저를 종료합니다.\n' + '=' * 100)
        time.sleep(0.7)
        browser.close()
        break

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]


크롤링을 시작합니다.


<ipython-input-15-12eb1facd1c3>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table = browser.find_element_by_xpath('//ul[@class="list_news"]')
C:\Users\dragew\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:392: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
 10%|████████                                                                         | 10/100 [00:06<00:49,  1.83it/s]<ipython-input-15-12eb1facd1c3>:30: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pages = browser.find_element_by_xpath('//div[@class="sc_page_inner"]')
100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:49<00:00,  2.01it/s]



브라우저를 종료합니다.


In [16]:
#### 데이터 전처리하기 ###################################################### 

print('데이터프레임 변환\n')
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_본문_{}개_{}_{}.xlsx'.format(news_num, query, date)

news_df.to_excel(xlsx_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, xlsx_file_name))

os.startfile(folder_path)

print('=' * 100 + '\n결과물의 일부')
news_df


데이터프레임 변환

엑셀 저장 완료 | 경로 : C:\Users\dragew\네이버뉴스_본문_100개_esg_2021-11-22_15시11분.xlsx

결과물의 일부


,title,url,text
1,한국언론진흥재단 “지속 가능 경영 위한 ESG경영 원년 선포”,https://news.kbs.co.kr/news/view.do?ncd=532155...,한국언론진흥재단이 지속 가능 경영을 위한 ESG경영 원년을 선포했다고 밝혔습니다. ...
2,기업 10곳 중 8곳 “개별기업 차원 넘어 공급망 ESG 협력 중요”,https://news.kbs.co.kr/news/view.do?ncd=530103...,국내 기업 10곳 중 8곳은 개별기업 차원을 넘어 납품·협력업체까지 포함한 ‘공급망...
3,“지역 중소기업 ‘ESG 경영’ 인식 부족”,https://news.kbs.co.kr/news/view.do?ncd=529253...,"대구경북 중소기업은 지속가능한 경영 요소인 환경과 사회,지배구조, 이른바 ESG에 ..."
4,"[충북&오늘] 진천군, 폭우 대응 강화…옥천군 ESG 평가 A등급",https://news.kbs.co.kr/news/view.do?ncd=529516...,[앵커] 충북 앤 오늘 순서입니다. 진천군이 침수와 홍수 위험에 신속하게 대응할 ...
5,ESG 경영 필요한데…중소기업 89% “준비 안 돼 있다”,https://news.kbs.co.kr/news/view.do?ncd=529069...,"국내 중소기업들이 환경·사회·지배구조(ESG) 경영의 필요성을 느끼지만, 도입 준비..."
...,...,...,...
96,"통신요금, 편리한 전자청구서로…“지구 건강 지킨다”",https://news.kbs.co.kr/news/view.do?ncd=527406...,한국정보통신진흥협회(KAIT)는 오늘(7일) 이동통신 3사와 함께 전자청구서 이용 ...
97,환불 재개된 머지포인트…향후 전망은 불투명,https://news.kbs.co.kr/news/view.do?ncd=525879...,[앵커] 대규모 환불 사태를 일으킨 머지포인트 관련 논란이 이번 주에도 계속되고...
98,환불 재개됐지만…‘머지 사태’는 진행형,https://news.kbs.co.kr/news/view.do?ncd=525804...,[앵커] 대규모 환불 사태를 일으킨 머지포인트 관련 논란이 이번 주에도 계속되고...
99,플라스틱 병뚜껑 활용 노인 일자리 사업 지원,https://news.kbs.co.kr/news/view.do?ncd=526478...,"울산시와 한국노인인력개발원, 시청자미디어재단 등이 기업의 환경·사회·지배구조를 뜻하..."
